## Importando Bibliotecas

In [37]:
import os
import sys
import pandas as pd
import numpy as np

#Plots
import matplotlib.pyplot as plt
import seaborn as sns

# `do not disturb` mode
import warnings                                  
warnings.filterwarnings('ignore')
                           
# Dates Manipulation
from dateutil.relativedelta import relativedelta 
from datetime import datetime, timedelta
import holidays
import calendar

# statistics
import statsmodels.formula.api as smf            
import statsmodels.tsa.api as smt
import statsmodels.api as sm

#Signal Processing
import scipy as sp
import scipy.fftpack
from scipy.optimize import minimize

In [38]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")

params = {'figure.figsize': [10, 12], 
          'axes.labelsize': 16,
          'axes.titlesize':18, 
          'font.size': 16,
          'legend.fontsize': 12, 
          'xtick.labelsize': 12, 
          'ytick.labelsize': 12
         }

plt.rcParams.update(params)


In [39]:
# Seta display de x linhas
pd.set_option('display.max_row', 100)
# Seta display de x colunas
pd.set_option('display.max_columns', 15)

## Funções e Configurações

In [40]:
#######  function [GRNN]=grnn_v1(p,t,spread) #######
def train_matrix(p, t, spread):
    #Cria Matriz de Treinamento: 
    
    # p -> Matriz contendo os vetores de entrada
    # t -> Matriz contendo os vetores de saída
    # spread -> valor do spread que será utilizado
    
    #Cria matriz com as matrizes de entrada p e t
    GRNN=np.zeros((1+t.shape[0]+p.shape[0],t.shape[1]))
    
    # Verifica o número de padrões utilizados no treinamento "n"
    if p.shape[1] == t.shape[1]: 
        # número de entradas do vetor de entrada "ne"
        GRNN[0,0]=p.shape[0]
        # número de saídas do vetor de saída "ns"
        GRNN[0,1]=t.shape[0]
        # Spread
        GRNN[0,2]=spread
        
        #vetores de entrada P
        GRNN[1:1+p.shape[0],:]=p
        #Vetores de saída T
        GRNN[1+p.shape[0]:1+p.shape[0]+t.shape[0],:]=t
    return GRNN;

### GRNN

In [41]:
####### Função que implementa a GRNN #######

def implementa_grnn(GRNN, X): 
    
    # GRNN - Saída da Função grnn_v1(p,t,spread) #
    # X    - Valor a ser aproximado #

    # Definição de ne, n
    sizep = [GRNN[0,0], GRNN.shape[1]] 
    
    # Definição de ns, n
    sizet = [GRNN[0,1], GRNN.shape[1]] 
    
    # Parâmetro spread
    spread = GRNN[0,2]

    
    p = GRNN[1:1+int(sizep[0]),:]
    t = GRNN[1+int(sizep[0]):1+int(sizep[0])+int(sizet[0]),:]

    c1 = np.zeros((int(sizep[0]),int(sizep[1])))
    c3 = np.zeros((1,int(sizep[1])))
    
    num = np.zeros((int(sizet[0]),int(sizet[1])))
    den = np.copy(c3)
    
    Y = np.zeros((int(sizet[0]),np.size(X,1)))
    A = np.zeros((int(sizep[0]),int(sizep[1])))

    if (p.shape[0] == X.shape[0]):
        for i in range(X.shape[1]):
            for k in range(int(sizep[0])):
                A[k,:] = X[k,i]*np.ones((1,int(sizep[1])))
            c1 = abs(A-p)**2
            # soma "MATRIZ"
            c2 = np.sqrt(c1.sum(axis=0)) 
            
    ########### Aqui é a GRNN ########### 

            for j in range(int(sizep[1])): 
                c3[0,j] = (1)*np.exp(-(0.8326*c2[j]/spread)**2)
                for k in range(int(sizet[0])):
                    num[k,j] = t[k,j]*c3[0,j]
                den[0,j] = c3[0,j]
                
    ########################################

            for k in range(int(sizet[0])):
                #Y(k,i)=sum(num(k,:)/(sum(den)+1e-9)
                Y[k,i]=(np.sum(num[k,:])/(np.sum(den)+0.0000000001))
    return Y

### GRNN Modificada

In [42]:
####### Função que implementa a GRNN Modificada #######

def implementa_grnn_mod(GRNN, X, nmax): 
    
    # GRNN - Saída da Função grnn_v1(p,t,spread) #
    # X    - Valor a ser aproximado #
    
    # Definição de ne, n
    sizep=[GRNN[0,0], GRNN.shape[1]]
    
    # Definição de ns, n
    sizet=[GRNN[0,1], GRNN.shape[1]] 
    
    # Parâmetro spread
    spread=GRNN[0,2]

    p=GRNN[1:1+int(sizep[0]),:]
    t=GRNN[1+int(sizep[0]):1+int(sizep[0])+int(sizet[0]),:]

    c1=np.zeros((int(sizep[0]),int(sizep[1])))
    c3=np.zeros((1,nmax))

    num=np.zeros((int(sizet[0]),int(sizet[1])))
    den=np.copy(c3)
    
    Y=np.zeros((int(sizet[0]), X.shape[1]))
    A=np.zeros((int(sizep[0]),int(sizep[1])))
    
    if (p.shape[0] == X.shape[0]):
        for i in range(X.shape[1]):      
            for k in range(int(sizep[0])):
                A[k,:]=X[k,i]*np.ones((1,int(sizep[1])))
            
            c1=abs(A-p)**2
            
            # soma "MATRIZ"
            c2=np.sqrt(c1.sum(axis=0))

########### Aqui é a GRNN modificada ###########
        
            ind_c2 = np.argsort(c2)
            ord_c2 = np.sort(c2)
        
            for j in range(nmax): 
                
                c3[0,j]=(1)*np.exp(-(0.8326*ord_c2[j]/spread)**2)
                
                for k in range(int(sizet[0])):
                    
                    num[k,j]=t[k,ind_c2[j]]*c3[0,j]
                    
                den[0,j]=c3[0,j]
                
####################################################
            
            for k in range(int(sizet[0])):
                Y[k,i]=(np.sum(num[k,:])/(np.sum(den)+0.0000000001))
    return Y

### Avaliação de Desempenho

In [43]:
def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [44]:
def mape(y_true, y_pred): 
    return np.max(np.abs((y_true - y_pred) / y_true)) * 100

### Pré Processamento

In [45]:
## Filtra os datasets para o ano de 2018

def filteryear(df):
    df = df[(df['year']) == 2018]
    return df

In [46]:
#Normaliza os valorews de carga

def normalize(df, new_column, column):
    df[new_column] = df[column]/max(df[column])

In [47]:
def extract_time(df):
    df['time'] = df['data_medicao'].dt.time

In [48]:
## Pega o mês de Referência

def get_month(df):
    df['inp_month'] = df['month']/12

In [49]:
## Pega dummies para os dias da semana

def getmon(df):
    
    if df['week_day'] == 'MONDAY' :
        return 1
    else :
        return 0
    
def gettue_res(df):    
    if df['week_day'] == 'TUESDAY' :
        return 1
    else :
        return 0
    
def getwed_res(df):    
    if df['week_day'] == 'WEDNESDAY' :
        return 1
    else :
        return 0

def getthu_res(df):       
    if df['week_day'] == 'THURSDAY' :
        return 1
    else :
        return 0
    
def getfri_res(df):       
    if df['week_day'] == 'FRIDAY' :
        return 1
    else :
        return 0
    
def getsat(df):       
    if df['week_day'] == 'SATURDAY' :
        return 1
    else :
        return 0
    
def getsun(df):           
    if df['week_day'] == 'SUNDAY' :
        return 1
    else :
        return 0
    
def getweek_ot(df):    
    if df['week_day'] == 'TUESDAY' or df['week_day'] == 'WEDNESDAY' or\
    df['week_day'] == 'THURSDAY' or df['week_day'] == 'FRIDAY':
        return 1
    else :
        return 0

In [50]:
def residential_stats(df):
    df['is_monday'] = df.apply (lambda row: getmon(row), axis=1)

    df['is_tuesday'] = df.apply (lambda row: gettue_res(row), axis=1)

    df['is_wednesday'] = df.apply (lambda row: getwed_res(row), axis=1)
    
    df['is_thursday'] = df.apply (lambda row: getthu_res(row), axis=1)
    
    df['is_friday'] = df.apply (lambda row: getfri_res(row), axis=1)
    
    df['is_saturday'] = df.apply (lambda row: getsat(row), axis=1)
    
    df['is_sunday'] = df.apply (lambda row: getsun(row), axis=1)

In [51]:
def other_stats(df):
    df['is_monday'] = df.apply (lambda row: getmon(row), axis=1)

    df['is_week'] = df.apply (lambda row: getweek_ot(row), axis=1)

    df['is_saturday'] = df.apply (lambda row: getsat(row), axis=1)
    
    df['is_sunday'] = df.apply (lambda row: getsun(row), axis=1)

In [52]:
def load_statistics(df, column1, column2):
    
    # Input: df receives dataframe and column receives string

    result = df.groupby(column1)[column2].agg(['min','max','mean'])
    shift_df = result.shift(periods=1, fill_value=0).reset_index()

    return shift_df

## Importando os Datasets

In [53]:
%%time
# Definindo os nomes das colunas
typing = {'instalacao': 'str'}

#importando o arquivo - Dataset de dados de carga Residenciais
data_res = pd.read_csv(r'C:/Users/vitmi/Desktop/TCC/Data/load_data/Data_res_mean.csv', encoding='utf-8', 
                       sep = ';', decimal = '.',  dtype = typing, parse_dates = ['data_medicao'])
data_res = filteryear(data_res)
get_month(data_res)
extract_time(data_res)
normalize(data_res, 'valor_kwh_n', 'valor_kwh')


#importando o arquivo - Dataset de dados de carga Comerciais
data_com = pd.read_csv(r'C:/Users/vitmi/Desktop/TCC/Data/load_data/data_com_mean.csv', encoding='utf-8', 
                       sep = ';', decimal = '.',  dtype = typing, parse_dates = ['data_medicao'])
data_com = filteryear(data_com)
get_month(data_com)
extract_time(data_com)
normalize(data_com, 'valor_kwh_n', 'valor_kwh')

#importando o arquivo - Dataset de dados de carga - Poder Público Municipal
data_ppm = pd.read_csv(r'C:/Users/vitmi/Desktop/TCC/Data/load_data/Data_ppm_mean.csv', encoding='utf-8', 
                       sep = ';', decimal = '.',  dtype = typing, parse_dates = ['data_medicao'])
data_ppm = filteryear(data_ppm)
get_month(data_ppm)
extract_time(data_ppm)
normalize(data_ppm, 'valor_kwh_n', 'valor_kwh')

#importando o arquivo - Dataset de dados de carga - Serviço Público - Água, Saneamento e Esgoto
data_sp = pd.read_csv(r'C:/Users/vitmi/Desktop/TCC/Data/load_data/Data_sp_mean.csv', encoding='utf-8', 
                       sep = ';', decimal = '.',  dtype = typing, parse_dates = ['data_medicao'])
data_sp = filteryear(data_sp)
get_month(data_sp)
extract_time(data_sp)
normalize(data_sp, 'valor_kwh_n', 'valor_kwh')


Wall time: 2.03 s


### Pré Processamento

In [54]:
data_res_group = pd.DataFrame(data_res.groupby(by = ['data_med', 'inp_month', 'hora_verao', 'feriado', 'week_day'])\
                                 ['valor_kwh'].mean()).reset_index(drop = False)
residential_stats(data_res_group)
data_res_group = data_res_group.drop(columns=['data_med', 'valor_kwh', 'week_day'])


data_com_group = pd.DataFrame(data_com.groupby(by = ['data_med', 'inp_month', 'hora_verao', 'feriado', 'week_day'])\
                                 ['valor_kwh'].mean()).reset_index(drop = False)
other_stats(data_com_group)
data_com_group = data_com_group.drop(columns=['data_med', 'valor_kwh', 'week_day'])


data_ppm_group = pd.DataFrame(data_ppm.groupby(by = ['data_med', 'inp_month', 'hora_verao', 'feriado', 'week_day'])\
                                 ['valor_kwh'].mean()).reset_index(drop = False)
other_stats(data_ppm_group)
data_ppm_group = data_ppm_group.drop(columns=['data_med', 'valor_kwh', 'week_day'])


data_sp_group = pd.DataFrame(data_sp.groupby(by = ['data_med', 'inp_month', 'hora_verao', 'feriado', 'week_day'])\
                                 ['valor_kwh'].mean()).reset_index(drop = False)
other_stats(data_sp_group)
data_sp_group = data_sp_group.drop(columns=['data_med', 'valor_kwh', 'week_day'])

In [55]:
data_res_stats = load_statistics(data_res, 'data_med', 'valor_kwh_n')
data_com_stats = load_statistics(data_com, 'data_med', 'valor_kwh_n')
data_ppm_stats = load_statistics(data_ppm, 'data_med', 'valor_kwh_n')
data_sp_stats = load_statistics(data_sp, 'data_med', 'valor_kwh_n')

### Matriz de entrada X

In [56]:
data_mod_res = pd.concat([data_res_group, data_res_stats], axis=1, join="inner").set_index(['data_med'])
data_mod_com = pd.concat([data_com_group, data_com_stats], axis=1, join="inner").set_index(['data_med'])
data_mod_ppm = pd.concat([data_ppm_group, data_ppm_stats], axis=1, join="inner").set_index(['data_med'])
data_mod_sp = pd.concat([data_sp_group, data_sp_stats], axis=1, join="inner").set_index(['data_med'])

#### Treino, Validação e Teste 

In [57]:
data_mod_res_tre = data_mod_res[(data_mod_res['inp_month']*12 < 9)]
data_mod_res_tre_m = np.matrix(data_mod_res_tre.T)

data_mod_res_val = data_mod_res[(data_mod_res['inp_month']*12 < 12) & (data_mod_res['inp_month']*12 > 8) ]
data_mod_res_val_m = np.matrix(data_mod_res_val.T)

data_mod_res_teste1 = data_mod_res.loc[(data_mod_res.index > '2018-11-30') & (data_mod_res.index < '2018-12-09')]
data_mod_res_teste1_m = np.matrix(data_mod_res_teste1.T)

data_mod_res_teste2 = data_mod_res.loc[(data_mod_res.index > '2018-12-22') & (data_mod_res.index < '2018-12-30')]
data_mod_res_teste2_m = np.matrix(data_mod_res_teste2.T)

In [58]:
data_mod_com_tre = data_mod_com[(data_mod_com['inp_month']*12 < 9)]
data_mod_com_tre_m = np.matrix(data_mod_com_tre.T)

data_mod_com_val = data_mod_com[(data_mod_com['inp_month']*12 < 12) & (data_mod_com['inp_month']*12 > 8) ]
data_mod_com_val_m = np.matrix(data_mod_com_val.T)

data_mod_com_teste1 = data_mod_com.loc[(data_mod_com.index > '2018-11-30') & (data_mod_com.index < '2018-12-09')]
data_mod_com_teste1_m = np.matrix(data_mod_com_teste1.T)

data_mod_com_teste2 = data_mod_com.loc[(data_mod_com.index > '2018-12-22') & (data_mod_com.index < '2018-12-30')]
data_mod_com_teste2_m = np.matrix(data_mod_com_teste2.T)

In [59]:
data_mod_ppm_tre = data_mod_ppm[(data_mod_ppm['inp_month']*12 < 9)]
data_mod_ppm_tre_m = np.matrix(data_mod_ppm_tre.T)

data_mod_ppm_val = data_mod_ppm[(data_mod_ppm['inp_month']*12 < 12) & (data_mod_ppm['inp_month']*12 > 8) ]
data_mod_ppm_val_m = np.matrix(data_mod_ppm_val.T)

data_mod_ppm_teste1 = data_mod_ppm.loc[(data_mod_ppm.index > '2018-11-30') & (data_mod_ppm.index < '2018-12-09')]
data_mod_ppm_teste1_m = np.matrix(data_mod_ppm_teste1.T)

data_mod_ppm_teste2 = data_mod_ppm.loc[(data_mod_ppm.index > '2018-12-22') & (data_mod_ppm.index < '2018-12-30')]
data_mod_ppm_teste2_m = np.matrix(data_mod_ppm_teste2.T)

In [60]:
data_mod_sp_tre = data_mod_sp[(data_mod_sp['inp_month']*12 < 9)]
data_mod_sp_tre_m = np.matrix(data_mod_sp_tre.T)

data_mod_sp_val = data_mod_sp[(data_mod_sp['inp_month']*12 < 12) & (data_mod_sp['inp_month']*12 > 8) ]
data_mod_sp_val_m = np.matrix(data_mod_sp_val.T)

data_mod_sp_teste1 = data_mod_sp.loc[(data_mod_sp.index > '2018-11-30') & (data_mod_sp.index < '2018-12-09')]
data_mod_sp_teste1_m = np.matrix(data_mod_sp_teste1.T)

data_mod_sp_teste2 = data_mod_sp.loc[(data_mod_sp.index > '2018-12-22') & (data_mod_sp.index < '2018-12-30')]
data_mod_sp_teste2_m = np.matrix(data_mod_sp_teste2.T)

### Matriz de Saída Y

In [61]:
data_out_res = pd.pivot_table(data_res, values = 'valor_kwh_n', columns='data_med', index = 'time')
data_out_com = pd.pivot_table(data_com, values = 'valor_kwh_n', columns='data_med', index = 'time')
data_out_ppm = pd.pivot_table(data_ppm, values = 'valor_kwh_n', columns='data_med', index = 'time')
data_out_sp = pd.pivot_table(data_sp, values = 'valor_kwh_n', columns='data_med', index = 'time')

#### Treino, Validação e Teste 

In [62]:
data_out_res_tre = data_out_res.T.loc[(data_out_res.T.index <= '2018-08-31')].T
data_out_res_tre_m = np.matrix(data_out_res_tre)

data_out_res_val = data_out_res.T.loc[(data_out_res.T.index <= '2018-11-30') & (data_out_res.T.index >= '2018-09-01')].T
data_out_res_val_m = np.matrix(data_out_res_val)

data_out_res_teste1 = data_out_res.T.loc[(data_out_res.T.index >= '2018-12-01') & (data_out_res.T.index <= '2018-12-08')].T
data_out_res_teste1_m = np.matrix(data_out_res_teste1)

data_out_res_teste2 = data_out_res.T.loc[(data_out_res.T.index >= '2018-12-23') & (data_out_res.T.index <= '2018-12-29')].T
data_out_res_teste2_m = np.matrix(data_out_res_teste2)

In [63]:
data_out_com_tre = data_out_com.T.loc[(data_out_com.T.index <= '2018-08-31')].T
data_out_com_tre_m = np.matrix(data_out_com_tre)

data_out_com_val = data_out_com.T.loc[(data_out_com.T.index <= '2018-11-30') & (data_out_com.T.index >= '2018-09-01')].T
data_out_com_val_m = np.matrix(data_out_com_val)

data_out_com_teste1 = data_out_com.T.loc[(data_out_com.T.index >= '2018-12-01') & (data_out_com.T.index <= '2018-12-08')].T
data_out_com_teste1_m = np.matrix(data_out_com_teste1)

data_out_com_teste2 = data_out_com.T.loc[(data_out_com.T.index >= '2018-12-23') & (data_out_com.T.index <= '2018-12-29')].T
data_out_com_teste2_m = np.matrix(data_out_com_teste2)

In [64]:
data_out_ppm_tre = data_out_ppm.T.loc[(data_out_ppm.T.index <= '2018-08-31')].T
data_out_ppm_tre_m = np.matrix(data_out_ppm_tre)

data_out_ppm_val = data_out_ppm.T.loc[(data_out_ppm.T.index <= '2018-11-30') & (data_out_ppm.T.index >= '2018-09-01')].T
data_out_ppm_val_m = np.matrix(data_out_ppm_val)

data_out_ppm_teste1 = data_out_ppm.T.loc[(data_out_ppm.T.index >= '2018-12-01') & (data_out_ppm.T.index <= '2018-12-08')].T
data_out_ppm_teste1_m = np.matrix(data_out_ppm_teste1)

data_out_ppm_teste2 = data_out_ppm.T.loc[(data_out_ppm.T.index >= '2018-12-23') & (data_out_ppm.T.index <= '2018-12-29')].T
data_out_ppm_teste2_m = np.matrix(data_out_ppm_teste2)

In [65]:
data_out_sp_tre = data_out_sp.T.loc[(data_out_sp.T.index <= '2018-08-31')].T
data_out_sp_tre_m = np.matrix(data_out_sp_tre)

data_out_sp_val = data_out_sp.T.loc[(data_out_sp.T.index <= '2018-11-30') & (data_out_sp.T.index >= '2018-09-01')].T
data_out_sp_val_m = np.matrix(data_out_sp_val)

data_out_sp_teste1 = data_out_sp.T.loc[(data_out_sp.T.index >= '2018-12-01') & (data_out_sp.T.index <= '2018-12-08')].T
data_out_sp_teste1_m = np.matrix(data_out_sp_teste1)

data_out_sp_teste2 = data_out_sp.T.loc[(data_out_sp.T.index >= '2018-12-23') & (data_out_sp.T.index <= '2018-12-29')].T
data_out_sp_teste2_m = np.matrix(data_out_sp_teste2)

### GRNN - Modelo Comercial

In [66]:
spread = 0.1

In [67]:
#data_mod_com_tre_m
#data_mod_com_val_m
#data_mod_com_teste1_m
#data_mod_com_teste2_m

In [68]:
#data_out_com_tre_m
#data_out_com_val_m
#data_out_com_teste1_m
#data_out_com_teste2_m

In [72]:
GRNN_com = train_matrix(data_mod_com_tre_m, data_out_com_tre_m, spread)

In [73]:
GRNN_com.shape

(59, 243)

In [ ]:
data_mod_com_val = data_mod_com[(data_mod_com['inp_month']*12 < 12) & (data_mod_com['inp_month']*12 > 8) ]
data_mod_com_val_m = np.matrix(data_mod_com_val.T)

In [75]:
data_com

,instalacao,valor_kwh,data_medicao,data_med,year,month,day,week_day,feriado,hora_verao,inp_month,time,valor_kwh_n
0,0000001261,36.800000,2018-01-01 00:00:00,2018-01-01,2018,1,1,MONDAY,1,0,0.083333,00:00:00,0.063845
1,0000001261,37.600000,2018-01-01 00:30:00,2018-01-01,2018,1,1,MONDAY,1,0,0.083333,00:30:00,0.065232
2,0000001261,36.800000,2018-01-01 01:00:00,2018-01-01,2018,1,1,MONDAY,1,0,0.083333,01:00:00,0.063845
3,0000001261,38.400000,2018-01-01 01:30:00,2018-01-01,2018,1,1,MONDAY,1,0,0.083333,01:30:00,0.066620
4,0000001261,38.400000,2018-01-01 02:00:00,2018-01-01,2018,1,1,MONDAY,1,0,0.083333,02:00:00,0.066620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324298,0000012017,17.173333,2018-12-31 21:30:00,2018-12-31,2018,12,31,MONDAY,0,1,1.000000,21:30:00,0.029794
324299,0000012017,13.276667,2018-12-31 22:00:00,2018-12-31,2018,12,31,MONDAY,0,1,1.000000,22:00:00,0.023034
324300,0000012017,12.366667,2018-12-31 22:30:00,2018-12-31,2018,12,31,MONDAY,0,1,1.000000,22:30:00,0.021455
324301,0000012017,12.600000,2018-12-31 23:00:00,2018-12-31,2018,12,31,MONDAY,0,1,1.000000,23:00:00,0.021860


In [71]:
implementa_grnn(GRNN_com, data_out_com_val_m)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])